# Analyse retail of transactions

1. Check double in data
2. How many transactions were canceled by users in total?
3. How many transactions were made by users in total?
4. How many transactions were made by users in Germany (using percentele)?
5. Which of the products was added to the cart most often, except for POST?
6. Calculate revenue per invoice

In [26]:
# We have data of transactions from 01.12.2010 to 12.09.2011:
# InvoiceNo — number of transactions
# StockCode — the product contact
# Description 
# Quantity 
# InvoiceDate — date of transaction
# UnitPrice 
# CustomerID — 
# Country — country of customer

In [15]:
import pandas as pd

###### 1. Check double in data

In [16]:
retail=pd.read_csv('5_data.csv.zip', compression='zip', encoding='ISO-8859-1')
retail_columns=retail.columns.values.tolist()
retail_columns

['InvoiceNo',
 'StockCode',
 'Description',
 'Quantity',
 'InvoiceDate',
 'UnitPrice',
 'CustomerID',
 'Country']

In [17]:
retail_columns_2var=retail.columns

In [18]:
retail_columns_2var

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [19]:
#other solution
len(retail) - len(retail.drop_duplicates())

5268

In [20]:
# Check the double.
# Delete dupl.

In [21]:
retail_dupl=retail.loc[retail.duplicated()]

In [22]:
retail_dupl.shape

(5268, 8)

In [23]:
retail=retail.drop_duplicates()

In [25]:
retail.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


###### 2. How many transactions were canceled by users in total?

In [177]:
retail.InvoiceNo.str[0].unique()

array(['5', 'C', 'A'], dtype=object)

In [178]:
retail.loc[retail.InvoiceNo.str[0] == 'C'].shape

(9251, 8)

###### 3. How many transactions were made by users in total?

In [37]:
# orders only where Quantity > 0.

In [38]:
retail.query('Quantity > 0').shape

(526054, 8)

###### 4. How many transactions were made by users in Germany (using percentele)? 

In [40]:
retail.loc[retail.Country == 'Germany'].shape

(9480, 8)

In [41]:
retail_in_Germany=retail.query('Country == "Germany"').groupby('CustomerID', as_index=False).InvoiceNo.nunique()

In [42]:
retail_in_Germany.head(5)

,CustomerID,InvoiceNo
0,12426.0,1
1,12427.0,5
2,12468.0,3
3,12471.0,49
4,12472.0,13


In [43]:
N=retail_in_Germany.InvoiceNo.quantile(0.8)

In [44]:
N

9.0

In [45]:
retail_in_Germany=retail.query('Country == "Germany"').groupby('CustomerID', as_index=False).InvoiceNo.nunique()
N=retail_in_Germany.InvoiceNo.quantile(0.8)
retail_N=retail_in_Germany.loc[retail_in_Germany.InvoiceNo > N]
germany_top=retail_N.CustomerID

In [46]:
germany_top

3     12471.0
4     12472.0
6     12474.0
8     12476.0
12    12481.0
16    12500.0
37    12569.0
44    12600.0
52    12619.0
53    12621.0
57    12626.0
65    12647.0
70    12662.0
78    12705.0
79    12708.0
80    12709.0
82    12712.0
84    12720.0
Name: CustomerID, dtype: float64

In [47]:
top_retail_germany = retail[retail.CustomerID.isin(germany_top)]

In [48]:
top_retail_germany.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1109,536527,22809,SET OF 6 T-LIGHTS SANTA,6,12/1/2010 13:04,2.95,12662.0,Germany
1110,536527,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,6,12/1/2010 13:04,2.55,12662.0,Germany
1111,536527,84945,MULTI COLOUR SILVER T-LIGHT HOLDER,12,12/1/2010 13:04,0.85,12662.0,Germany
1112,536527,22242,5 HOOK HANGER MAGIC TOADSTOOL,12,12/1/2010 13:04,1.65,12662.0,Germany
1113,536527,22244,3 HOOK HANGER MAGIC GARDEN,12,12/1/2010 13:04,1.95,12662.0,Germany


###### 5.Which of the products was added to the cart most often, except for POST?

Note: one order is considered a one-time purchase of any quantity of goods, i.e. excluding Quantity.

In [49]:
top_retail_germany.Description.unique()

array(['SET OF 6 T-LIGHTS SANTA', 'ROTATING SILVER ANGELS T-LIGHT HLDR',
       'MULTI COLOUR SILVER T-LIGHT HOLDER', ...,
       'BALLOON ART MAKE YOUR OWN FLOWERS', 'VINTAGE BELLS GARLAND',
       'SET OF 6 RIBBONS PERFECTLY PRETTY  '], dtype=object)

In [51]:
toptop=top_retail_germany.groupby(['StockCode','Description'], as_index=False)\
                         .agg({'InvoiceNo':'count'})\
                         .sort_values('InvoiceNo', ascending=False)

In [52]:
toptop

,StockCode,Description,InvoiceNo
1188,POST,POSTAGE,219
413,22326,ROUND SNACK BOXES SET OF4 WOODLAND,62
457,22423,REGENCY CAKESTAND 3 TIER,55
415,22328,ROUND SNACK BOXES SET OF 4 FRUITS,45
479,22467,GUMBALL COAT RACK,37
...,...,...,...
600,22702,BLACK AND WHITE CAT BOWL,1
76,20997,ROSE DU SUD CUSHION COVER,1
604,22707,WRAP MONSTER FUN,1
605,22709,WRAP WEDDING DAY,1


In [53]:
# other solution

In [54]:
toptop=top_retail_germany.groupby('StockCode').size().sort_values(ascending=False)

In [55]:
toptop

StockCode
POST     219
22326     62
22423     55
22328     45
22467     37
        ... 
23008      1
22999      1
20717      1
22119      1
22821      1
Length: 1168, dtype: int64

###### 6.Calculate revenue per invoice

In [56]:
retail['Revenue']=retail.Quantity * retail.UnitPrice

In [58]:
retail.head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,22.00


In [59]:
top_5revenue=retail.groupby('InvoiceNo', as_index=False).agg({'Revenue':'sum'}).sort_values('Revenue', ascending=False).head(5)

In [60]:
top_5revenue

,InvoiceNo,Revenue
22025,581483,168469.60
2303,541431,77183.60
18776,574941,52940.94
19468,576365,50653.91
9741,556444,38970.00


In [61]:
top_5revenue.InvoiceNo.index

Int64Index([22025, 2303, 18776, 19468, 9741], dtype='int64')